# Broadening Mechanisms and Motional Narrowing 

In lecture, we learned about two different sources of broadening in molecular spectra: homogeneous broadening (associated with *differences* between molecular sites in a sample) and inhomogeneous broadening (associated with phenomena like electron-vibrational coupling and excited-state decay that affect all molecules similarly). In reality, these two types of broadening are opposite (idealized) limits in a continuum of broadening mechanisms associated with different dynamical time scales in a molecular system. This computational lab examines the transition from inhomogeneous to homogeneous broadening, introducing along the way the important concept of **motional narrowing**. 

As a simple illustration of the dynamical interplay between homogeneous and inhomogeneous broadening, let's return to the example we gave in the last lecture of hydrogen bond-induced inhomogeneous broadening. We noted in the last lecture that vibrational frequencies are often affected by hydrogen bonding: for example, C=O groups forming stronger hydrogen bonds tend to absorb light at lower frequencies, giving rise to an *inhomogeneous* broadening in their absorption spectra due to the distribution of hydrogen bonding strengths in a given system at any given time. 

But in a liquid sample, hydrogen bonds are constantly changing due to the evolution of the solvent environment. Thus a C=O group that was strongly hydrogen bonded at one moment may be only weakly bonded the next. In the extreme case, we might imagine a solvent environment that fluctuates on a timescale *faster* than the C=O oscillation period. In this case, it doesn't make sense to think about a static (inhomogneous) distribution of vibrational frequencies since the frequency changes *faster* than the bond itself vibrates! In fact, whatever spectral broadening the solvent might induce in this case seems much more reasonable to describe as *homogeneous* broadening, since the rapid cycling through high- and low-frequency configurations happens similarly for all sites in the sample. 

The simulation app below illustrates how absorption spectra are affected by frequency fluctuations on different time scales. Just like in our previous simulations, we'll watch how an ensemble of (harmonic) oscillators interact with a laser pulse. But this time, the *frequency* of each oscillator changes with time, mimicking different broadening mechanisms. The controls are similar to those you've used previously, but include two additional widgets: 
* The radio button that toggles between "Binary" and "Normal" sets the distribution function shape for the oscillator frequencies. When "Binary" is selected, each oscillator adopts one of exactly two possible frequencies. When "Normal" is selected, the frequencies are chosen from a normal (Gaussian) distribution. 
* The slide bar for $\\tau_{switch}$ controls the time scale over which oscillator frequencies change. The simulation is designed so that at each time-step, each oscillator has a chance to change its oscillation frequency. The probability of a frequency "jump" is controlled by $\\tau_{switch}$ so that, on average, frequency changes occur every $\\tau_{switch}$ fs. 

After the simulation runs, the app will plot a frequency trajectory for *one* selected oscillator, just to give you a sense for how the oscillator frequencies change with time. (Note that the frequency trajectory for each oscillator is different and randomly generated). Below the frequency trajectory, the absorption spectrum is plotted, along with the SDF (site distribution function) for the oscillator frequencies. The output text file contains a frequency axis in the first colum, the absorption spectrum in the second colum, and the SDF in the third column. 

## Instructions

For your homework submission, you'll run simulations with the app, read the text-file output into a new jupyter notebook file, and answer questions based on the simulation results. Be sure to submit both the ipynb file **and** the text files that store your simulation data. Without the text files I won't be able to run your code!

In [9]:
import math
%matplotlib inline
import matplotlib.pyplot as plt
import time
import ipywidgets as widgets
import matplotlib as mpl
mpl.rcParams['font.size'] = 16
import numpy as np

from IPython.display import Javascript, display
from ipywidgets import widgets
import IPython.display as ipd



def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))

gamma_slider = widgets.FloatSlider(
    value=10,
    min=0,
    max=150,
    step=1.0,
    description='$\gamma$ (pg/s):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.0f',
)
    
temp_slider = widgets.FloatSlider(
    value=0,
    min=0,
    max=1000,
    step=1.0,
    description='T (K):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.0f',
)

emax_slider = widgets.FloatSlider(
    value=100,
    min=0,
    max=250,
    step=1.0,
    description='$E_{max}\cdot 10^{-4}$ (statV/cm):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.0f',
)

tmax_slider = widgets.FloatSlider(
    value=3,
    min=0,
    max=10,
    step=1.0,
    description='$t_{max}$ (ps)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.0f',
)

tpulse_slider = widgets.FloatSlider(
    value=0.5,
    min=0,
    max=10,
    step=0.5,
    description='$t_{pulse}$ (ps)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

taupulse_slider = widgets.FloatSlider(
    value=20,
    min=10,
    max=1000,
    step=0.5,
    description='$\\tau_{pulse}$ (fs)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

vpulse_slider = widgets.FloatSlider(
    value=50.0,
    min=0,
    max=100,
    step=0.1,
    description='$\\nu_{pulse}$ (THz)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)



nmol_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=500,
    step=1,
    description='N$_{mol}$',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)


skip_box = widgets.Checkbox(
    value = True,
    description = 'Skip frames?'
)


txt_fname = widgets.Text(
    value='test.txt',
    placeholder='test.txt',
    description='Output File:',
    disabled=False
)


sdf_bt = widgets.RadioButtons(
    options=['Binary', 'Normal'],
    value = 'Binary',
    description='SDF type:',
    disabled=False
)


tauswitch_slider = widgets.IntSlider(
    min=1,
    value=1000,
    max=1000,
    step=1,
    description='$\\tau_{switch}$ (fs)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.0f',
)



button = widgets.Button(description="Go!")
button.on_click(run_all)

pulse_box = widgets.VBox([tpulse_slider, taupulse_slider, vpulse_slider, emax_slider])
part_box = widgets.VBox([nmol_slider, tmax_slider, gamma_slider, temp_slider])

display(widgets.HBox([pulse_box, part_box]))
display(widgets.HBox([tauswitch_slider, sdf_bt]))
display(skip_box)
display(button)
display(txt_fname)

Checkbox(value=True, description='Skip frames?')

Button(description='Go!', style=ButtonStyle())

Text(value='test.txt', description='Output File:', placeholder='test.txt')

<IPython.core.display.Javascript object>

In [1]:
gamma = gamma_slider.value*1e-12  # grams/second
Temp = temp_slider.value          # K
Emax = emax_slider.value*1e+4   # Maximum electric field in statV/cm
tmax = tmax_slider.value*1e-12      # Total simulation time in seconds
tpulse = tpulse_slider.value*1e-12      # Pulse arrival time in seconds
tau_pulse = taupulse_slider.value*1e-15  # Pulse width (std. dev.) in seconds
Nmol = nmol_slider.value
dt=0.5e-15     # Time-step in seconds
tmax = tmax_slider.value*1.0e-12
ofname = txt_fname.value

tau_switch = tauswitch_slider.value*1.0e-15

vpulse = vpulse_slider.value*1e+12                      # pulse frequency in Hz
skip_frames = skip_box.value

sdf_mode = sdf_bt.value

NameError: name 'gamma_slider' is not defined

In [ ]:
def calc_accel(x,y,efield):
    Minv = 1.0/M
    ax = np.zeros(np.shape(x))
    ay = np.zeros(np.shape(y))

    # X differences: Rx[m,n] is the x-displacement between particles m and n
    Rx = x - x.transpose()
    
    # Y differences: Ry[m,n] is the y-displacement between particles m and n
    Ry = y - y.transpose()
    
    dX = np.diag(Rx[0:Npos,Npos:])
    dY = np.diag(Ry[0:Npos,Npos:])
    dR = np.sqrt(np.power(dX,2) + np.power(dY,2))
    
    ax[0:Npos,:] -= np.reshape(K*Minv*(dR-Rbond)*(dX/dR), (Npos,1))/2.0
    ay[0:Npos,:] -= np.reshape(K*Minv*(dR-Rbond)*(dY/dR), (Npos,1))/2.0
    
    ax[Npos:,:] += np.reshape(K*Minv*(dR-Rbond)*(dX/dR), (Npos,1))/2.0
    ay[Npos:,:] += np.reshape(K*Minv*(dR-Rbond)*(dY/dR), (Npos,1))/2.0
    
    ay += np.reshape(Q*efield*Minv, (Npart,1))
    return ax,ay
    
def vv_step(x,y,vx,vy,ax,ay,efield):
    axrand = math.sqrt(2.0*kB*Temp*gamma/dt)*np.random.normal(0,1,(Npart,1))/M
    ayrand = math.sqrt(2.0*kB*Temp*gamma/dt)*np.random.normal(0,1,(Npart,1))/M
    xnew = x + B*dt*vx + 0.5*B*dt*dt*(ax + axrand)
    ynew = y + B*dt*vy + 0.5*B*dt*dt*(ay + ayrand)
    axnew,aynew = calc_accel(xnew,ynew,efield)
    vxnew = A*vx + 0.5*dt*(A*ax + axnew + 2.0*B*axrand)
    vynew = A*vy + 0.5*dt*(A*ay + aynew + 2.0*B*ayrand)
    
    # If the COM for a molecule has drifted out of the box, we want to move
    # the whole molecule back. The COM for each particle's molecule is 
    comx = 0.5*(x + np.fft.fftshift(x))
    comy = 0.5*(y + np.fft.fftshift(y))
    
    # Now if com < L we add L. If com>L, we subtract L:
    ynew += (comy<0)*L - (comy>L)*L
    xnew += (comx<0)*L - (comx>L)*L

    return xnew,ynew,vxnew,vynew,axnew,aynew

##############################
##############################

def init_plot():
    
    fig = plt.figure(1)
    ax1 = plt.gca()
    plt.title('Simulation box')
    
    bondLines = []
    for m in range(0, Nmol):
        bondLine, = plt.plot([X[m],X[Npos+m]], [Y[m],Y[Npos+m]],'k-')
        bondLines.append(bondLine)
        
    txt = plt.text(0.1*L,0.1*L,'t = '+str(round(0))+' fs')
    negLine, = plt.plot(X[0:Npos],Y[0:Npos],'bo')
    posLine, = plt.plot(X[Npos:],Y[Npos:],'ro')
    
    
    plt.xlim([0,L])
    plt.ylim([0,L])
    plt.xticks([])
    plt.yticks([])
    
    ax2 = plt.axes([1.0,0.575,0.3,0.3])
    plt.xlabel('$t$')
    plt.ylabel('$E(t)$')
    field_line, = plt.plot(taxis,Efield)
    plt.xticks([])
    plt.yticks([])
    plt.ylim([-1.1,1.1])
#     plt.ylim([0,dt*Nsteps])
    
    ax3 = plt.axes([1.0,0.175,0.3,0.3])
    plt.xlabel('$t$')
    plt.ylabel('$P(t)$')
    pol_line, = plt.plot(taxis,Pol)
    plt.xticks([])
    plt.yticks([])
    plt.ylim([-1,1])
    
    return fig,ax1,ax2,ax3,negLine,posLine,txt,field_line,pol_line,bondLines

def update_plot(n):
    negPts.set_ydata(Y[0:Npos])
    negPts.set_xdata(X[0:Npos])
    posPts.set_ydata(Y[Npos:])
    posPts.set_xdata(X[Npos:])
    txt.set_text('t = '+str(round(n*dt*1e+15))+' fs')
    
    for m in range(0, Nmol):
        bondLines[m].set_xdata([X[m],X[Npos+m]])
        bondLines[m].set_ydata([Y[m],Y[Npos+m]])
    
    plt.sca(ax2)
    field_line.set_xdata(taxis[0:n])
    field_line.set_ydata(Efield[0:n]/Emax)
    
    plt.sca(ax3)
    pol_line.set_xdata(taxis[0:n])
    pol_line.set_ydata(Pol[0:n])
    
    fig.canvas.draw()
    display(plt.gcf())
    ipd.clear_output(wait=True)
    
def gauss_pulse(t):
    return Emax*np.cos(2.0*math.pi*(t-to)*nu)*np.exp(-((t-to)**2)/(2.0*sigma*sigma))

epso = 190.0*(1.38064852e-23)*(1e+3)*(1e+4)*0.01
Ro = 10.0e-8

L = 0.25e-6
to = tpulse
sigma = tau_pulse
nu = vpulse

Nsteps=int(round(tmax/dt))
M=12*(1.66054e-24)   # Mass in g
Qo = 4.803e-10         # Elementary charge in statCoulombs
taxis = np.arange(0,tmax,dt)   # Time axis (array of time steps)

#####################################################
######### Constants for Langevin integrator #########
#####################################################

kB = 1.38064852e-16                  # erg/K
B = 1.0/(1.0 + 0.5*gamma*dt/M)
A = B*(1.0 - 0.5*gamma*dt/M)

#####################################################
#####################################################

# Set Particle charges
Npart = 2*Nmol

# One positive particle per molecule
Npos = Nmol
Q = np.zeros((Npart))  # Empty vector for particle charges
Q[0:Npos] = +Qo    # First Npos particles are positive
Q[Npos:] = -Qo     # Last Nneg particles are negative

# QQ is an (Npart)x(Npart) matrix where QQ[m,n] is the 
# *product* of the charges on particles m and n
QQ = np.reshape(Q, (Npart,1))@np.reshape(Q, (1,Npart))

# Set QQ elements that correspond to intermolecular bonds to zero
np.fill_diagonal(QQ[0:Npos,Npos:].view(), 0)
np.fill_diagonal(QQ[Npos:,0:Npos].view(), 0)


# Set initial velocities to zero 
VX = np.zeros((Npart,1))
VY = np.zeros((Npart,1))

# Generate the pulse profile
Efield = gauss_pulse(taxis)

# Frequency parameters
wcenter = 6.28*50.0e12
sigw = 0.025*wcenter

# Oscillator (angular) frequency
if sdf_mode=='Binary':
    wo = wcenter + sigw*(np.round(np.random.random((Npos,)))-0.5)
else:
    wo = np.random.normal(wcenter, sigw, (Npos,))
K = M*wo**2
Rbond = 1.24e-8


AX = 1e+50
AY = 1e+50
# while max(np.max(np.abs(AX)), np.max(np.abs(AY)))>1.0e+22:

# Generate a vector of random *molecular* positions
Rx = np.random.random((Nmol))*L
Ry = np.random.random((Nmol))*L

# Generate random orientations between 0 and 2*pi radians
Theta = np.random.random((Nmol))*2.0*math.pi

# Generate positions for positive and negative atoms in each molecule
# Note: X[n] and X[n+Npos] are bonded pairs! (Same with Y[n] and Y[n+Npos].)
X = np.concatenate((Rx+0.5*Rbond*np.cos(Theta),Rx-0.5*Rbond*np.cos(Theta)))
Y = np.concatenate((Ry+0.5*Rbond*np.sin(Theta),Ry-0.5*Rbond*np.sin(Theta)))
X.shape = (Npart,1)
Y.shape = (Npart,1)

# Calculate accelerations at initial positions
AX,AY = calc_accel(X,Y,Efield[0])

if skip_frames:
    UpdateFreq = 499
else:
    UpdateFreq = 20

wo_traj = np.zeros((Nsteps,))
pswitch = 2.0*dt/tau_switch
Pol = np.zeros(np.shape(Efield))
fig,ax1,ax2,ax3,negPts,posPts,txt,field_line,pol_line,bondLines = init_plot()
for n in range(0,Nsteps):
    X,Y,VX,VY,AX,AY = vv_step(X,Y,VX,VY,AX,AY,Efield[n])
    Pol[n] = 25*np.mean(Q*np.reshape(Y-0.5*L,(Npart,)))/(4*Qo*Rbond)
    if(n%UpdateFreq==0):
        update_plot(n)
    
    r = np.random.random((Npos,))
    
    switch_ndcs = np.where(r<pswitch)[0]
    if sdf_mode=='Binary':
        new_freqs = wcenter + sigw*(np.round(np.random.random(np.shape(switch_ndcs)))-0.5)
    else:
        new_freqs = np.random.normal(wcenter, sigw, np.shape(switch_ndcs))
    if len(new_freqs)>0:
        wo[switch_ndcs] = new_freqs
        K = M*wo**2
    wo_traj[n] = wo[0]

Efield /= np.max(Efield)
Pol /= np.max(Pol)

In [1]:
plt.plot(taxis*1e+12, 1e-12*wo_traj/(2.0*math.pi))
plt.xlabel('Time (ps)')
plt.ylabel('Frequency (THz)')
plt.title('Frequency Trajectory (single oscillator)')
plt.show()

fE = np.fft.ifft(Efield)
fP = np.fft.ifft(Pol)

faxis = np.fft.fftfreq(len(taxis), taxis[1] - taxis[0])

f1 = 40e+12
f2 = 60e+12

ndx1 = np.argmin(np.abs(faxis-f1))
ndx2 = np.argmin(np.abs(faxis-f2))

chi1 = fP[ndx1:ndx2]/fE[ndx1:ndx2]

if sdf_mode=='Binary':
    sdf = 0.0*taxis
    n1 = np.argmin(np.abs(faxis-(wcenter-0.5*sigw)/(2.0*math.pi)))
    sdf[n1] = 1.0
    n2 = np.argmin(np.abs(faxis-(wcenter+0.5*sigw)/(2.0*math.pi)))
    sdf[n2] = 1.0
else:
    sdf = np.exp(-(faxis-wcenter/(2.0*math.pi))**2/(2.0*(sigw/(2.0*math.pi))**2))
    sdf /= np.max(sdf)

A = np.imag(chi1)/np.max(np.abs(np.imag(chi1)))
plt.plot(faxis[ndx1:ndx2]*1e-12, A, label='Absorption')
plt.plot(faxis[ndx1:ndx2]*1e-12, sdf[ndx1:ndx2], color=[0.5,0.5,0.5], label='SDF')
plt.yticks([])
plt.legend()
plt.xlabel('Frequency (THz)')
plt.show()

np.savetxt("../../../../local/" + ofname, np.vstack([faxis[ndx1:ndx2], A.T, sdf[ndx1:ndx2]]).T)

NameError: name 'plt' is not defined

In [ ]:
import ipywidgets as widgets
import os
from IPython.display import display
from IPython.display import display_markdown

def copy_exercise(self):
    uname = txt_uname.value.replace(" ", "_").lower()
    #fpath = "~/MOLSPEC/local/"
    fpath = "../../../../local/"
    fname = "exercise5_" + uname + ".ipynb"
    
    if len(uname)<=0:
        print('Please enter a valid user name!')
    elif os.path.isfile(fpath+fname) and cb_overwrite.value==False:
        print('The file already exists! To overwrite check the \"Overwrite Existing\" box and try again.')
        FancyText = "Click [here](" + fpath + fname + ") to open existing copy."
        display_markdown(FancyText, raw=True)
    else:
        out = !{"cp exercise5.ipynb " + fpath+fname}
        if len(out)>0:
            for line in out:
                print(out)
        else:
            FancyText = "Successfully copied exercise to local directory!<br> Click [here](" + fpath + fname + ") to open."
            display_markdown(FancyText, raw=True)
    
txt_uname = widgets.Text(
    value='',
    placeholder='User name',
    description='Purdue ID:',
    disabled=False
)

bt_genfile = widgets.Button(
    description='Copy Exercise',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Enter your username and then click to create a local exercise file'
)

cb_overwrite = widgets.Checkbox(
    value=False,
    description='Overwrite Existing?',
    disabled=False
)

bt_genfile.on_click(copy_exercise)

display(widgets.HBox([txt_uname, bt_genfile, cb_overwrite]))